This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_predicted_vs_true_bars(y_true, y_pred_list, avg_stats,
                                savepath, title='best worst with bars', label='target_value'):
    """
    Method to calculate parity plot (predicted vs. true) of average predictions, averaged over all CV splits, with error

    bars on each point corresponding to the standard deviation of the predicted values over all CV splits.

    Args:

        y_true: (numpy array), array of true y data

        y_pred_list: (list), list of numpy arrays containing predicted y data for each CV split

        avg_stats: (dict), dict of calculated average metrics over all CV splits

        savepath: (str), path to save plots to

        title: (str), title of the best_worst_per_point plot

        label: (str), label used for axis labeling

    Returns:

        None

    """
    means = [nice_mean(y_pred) for y_pred in y_pred_list]
    standard_error_means = [nice_std(y_pred)/np.sqrt(len(y_pred))
                            for y_pred in y_pred_list]
    standard_errors = [nice_std(y_pred) for y_pred in y_pred_list]
    # make fig and ax, use x_align when placing text so things don't overlap
    x_align = 0.64
    fig, ax = make_fig_ax(x_align=x_align)

    # gather max and min
    max1 = max(np.nanmax(y_true), np.nanmax(means))
    min1 = min(np.nanmin(y_true), np.nanmin(means))

    # draw dashed horizontal line
    ax.plot([min1, max1], [min1, max1], 'k--', lw=2, zorder=1)

    # set axis labels
    ax.set_xlabel('True '+label, fontsize=16)
    ax.set_ylabel('Predicted '+label, fontsize=16)

    # set tick labels
    #maxx, minn = recursive_max_and_min([means, y_true])
    maxx = max(y_true)
    minn = min(y_true)
    maxx = round(float(maxx), rounder(maxx-minn))
    minn = round(float(minn), rounder(maxx-minn))
    #print(maxx, minn, rounder(maxx - minn))
    _set_tick_labels(ax, maxx, minn)

    ax.errorbar(y_true, means, yerr=standard_errors, fmt='o', markerfacecolor='blue', markeredgecolor='black', markersize=10,
                alpha=0.7, capsize=3)

    plot_stats(fig, avg_stats, x_align=x_align, y_align=0.90)

    fig.savefig(savepath, dpi=DPI, bbox_inches='tight')


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
y_true = array([177.28302038, 150.37102595, 202.0924573 , ...,  47.20790294,
        23.809066  ,   0.        ])
y_pred_list = [[227.47526250954073], [162.3130594483778], [140.78376572445907], [282.17900253031326], [302.16534380398025], [184.04045164304452], [206.05228516986674], [179.37758812435163], [201.71628658044304], [196.60601789386862], [232.91797473028907], [173.48205617293047], [167.53798764203648], [174.75324670225226], [168.2207603675778], [170.4800103494044], [174.15925854561507], [156.3512637536951], [156.7975807540852], [161.37046127414635], [157.1445329666306], [155.23379048874384], [300.71093431888613], [275.0129136317846], [284.0610135916593], [252.20241264989068], [277.70255442040315], [270.68353701726176], [210.74855313842247], [315.8130351488361], [251.34531134833028], [215.80162381082692], [141.47335689045457], [103.75390390606451], [103.44038950885155], [199.89702609295932], [174.67663800379754], [162.29084550104383], [120.40482433836891], [160.27351019307014], [115.29901521572027], [109.59768340315304], [127.73909147069851], [117.15247924011187], [95.61503357667465], [122.93111918397615], [91.06454233745372], [90.71974211086713], [110.28563768862534], [86.50511966277803], [94.06058632639487], [92.40593755884484], [92.03690397211906], [89.20120324729774], [238.6757127672011], [157.2327642505577], [156.61346009345777], [151.80840975345217], [154.56576262614152], [151.78921701733594], [199.70894346358276], [166.76363559693823], [139.89417750133703], [296.5283610464529], [170.27718983268193], [122.56285721025309], [96.73799877475561], [194.56955857533168], [177.07872104343363], [140.02305415038316], [161.51341459533992], [136.30564855095173], [149.82918711400512], [148.6712944697015], [163.4588387445456], [121.17304268957702], [121.6223796737885], [139.42540418343737], [101.04272836203705], [103.01935423516413], [127.25911439167491], [98.48248916030651], [93.40500926749019], [108.64690637687401], [88.17732443246962], [85.040055411943], [237.97472845499507], [125.57073042540866], [176.02823802272596], [123.81081370249741], [174.8071466160997], [174.68752735139603], [196.06754235454525], [175.56180553678962], [158.96970740302228], [153.21293953462705], [223.62894262478326], [164.6940442685421], [152.3637489078427], [294.6253966661982], [289.40656984950607], [177.2654697089428], [193.76213403713544], [175.18990648330998], [194.03521816021527], [189.49667684844002], [218.06152393813446], [171.5002281572071], [157.46966634909484], [174.4508361160888], [160.26374932556737], [156.43646167523266], [171.69415888080655], [147.4052815463743], [149.32764065590158], [154.40845087428872], [151.29429888251218], [147.50905376925027], [326.46848812693895], [244.65473062434117], [268.2379488936879], [227.0960871466019], [266.0982850871949], [260.3726030902124], [304.19758315681855], [233.06358840295388], [197.44109310018956], [323.6448420358082], [136.5600738049403], [100.94055676243383], [100.58716123242061], [190.93946502307466], [170.14482622107042], [156.81958427905457], [116.222982259577], [155.60326197015274], [109.00910402868442], [107.24348616931427], [123.26776580471684], [113.86308275731368], [105.18882189218255], [126.74987953683245], [88.66778979310747], [86.6143617382818], [106.3461455758993], [73.39242574450994], [91.65224025662765], [85.50728215923276], [89.40812435855652], [86.96562186759505], [223.68959581697652], [158.3187393502824], [153.46666439223083], [152.648660866734], [149.0915780591337], [147.15161726076363], [192.49984048814295], [164.6544938415282], [134.85955879270784], [285.4919342368002], [34.77309328371304], [41.639123526347326], [73.53643268150327], [88.07118562563215], [50.00377611779428], [31.975481062428663], [107.54653420897455], [193.16207409491057], [88.838092970005], [45.253926609977036], [32.526590953427295], [25.82640693543552], [27.35672354545873], [58.65500989595688], [59.98941888835705], [49.72872628510632], [46.75040291363702], [17.477176944516376], [100.5158042069376], [164.4090714463221], [115.2062603942708], [129.6425357368788], [130.61268004683794], [134.70898723095118], [123.5987087231914], [80.14567436472363], [66.2678154053313], [309.4069994222055], [77.53554405667019], [88.33658024438485], [119.78013249338666], [154.77960690257694], [184.61204026093552], [76.52809344692217], [78.15070297324328], [104.05578367393855], [94.74512679261885], [307.34944939764387], [78.84459219902797], [93.65644866574587], [121.11198730698693], [196.66277310473666], [88.54936531495339], [102.74487966173551], [103.81146311509262], [115.22313508721443], [71.33049443806047], [94.57557473730209], [93.23337548228126], [95.61584772689775], [89.76311189570009], [597.866912970315], [172.84677982941682], [232.8471007998556], [396.4251154979235], [503.6743188346251], [81.33631532507475], [80.8220946169584], [65.88155866691699], [74.44796399503106], [79.7512675044674], [83.51132263180008], [66.27535777029657], [103.25629422282054], [77.51093943210512], [69.89954000787075], [91.46385608275791], [101.7410774722544], [97.4790335592019], [124.88166472135333], [121.31611153747369], [115.24649884981646], [105.4160526786305], [162.1429680874751], [134.18876803615157], [146.00924695243862], [161.230552332089], [141.95745029859796], [112.57865258070184], [285.57330858588995], [132.9967741454196], [135.2508719219718], [170.5150131096266], [231.02368736104012], [174.29892911678076], [151.00392108901463], [164.69666105082274], [178.44832948011282], [173.8628352229812], [7.98184735594016], [65.6952446694397], [75.3028896940273], [25.88184885552401], [107.50398066452934], [103.74858566185048], [107.47818149082983], [95.61795704838501], [429.4874623486847], [124.00217545687438], [124.03861654234335], [185.14911270923304], [18.01815683328507], [92.10436373237613], [87.00052221847714], [94.44503490023807], [81.39634697474389], [107.87905858361611], [116.57700779453317], [209.64027798962175], [133.27710108991198], [132.43687755090608], [143.83816871417122], [111.60048554089909], [198.31728708921617], [185.6381592059818], [136.53822788211687], [134.04838009148705], [71.41964388140396], [93.98799011279992], [122.47536147213513], [199.87639800105154], [148.99427550055586], [147.99846170013586], [148.2716911228644], [139.6854842934644], [112.04729331497954], [108.79971901407576], [106.38725955607535], [45.54687644306507], [66.70535783216933], [75.44868374193248], [17.639913641314216], [109.89854857798908], [91.69876478457464], [82.52880246240062], [84.6166047387986], [154.34476376086704], [114.58688940832555], [117.56842321615642], [113.8853253930659], [290.3304705386676], [142.13753544765402], [154.1883701204302], [200.11571499284932], [249.88874101329844], [16.934030124491365], [96.1762172377958], [93.13672310136153], [82.7332532649646], [64.09111733451499], [37.45476622122382], [26.710080481760276], [43.255046216185264], [56.2447830193567], [106.99085733791202], [29.21022826855447], [67.04707827835966], [155.8168091799706], [56.09899467474804], [35.9760316166131], [10.41607079237442], [33.10495260730201], [47.80570125961207], [125.10356345481199], [29.957964732215252], [32.94400205930904], [37.43493247857763], [47.36339964216004], [66.23844044754742], [156.8024822060475], [56.15600103139535], [30.802177837603487], [3.9333209908419717], [47.2863028066302], [30.090580274863072], [61.24478186324176], [145.12945293054509], [88.00217508263937], [60.68029803170453], [75.44305037071818], [138.3846325718345], [76.99852797078778], [52.81165785469956], [85.64961839801106], [122.9512830252701], [181.2617303568675], [139.83568214264145], [50.464301745960206], [40.16245292335737], [164.81668751474268], [123.35606534731679], [136.4935424993273], [154.88389257232666], [131.475453564898], [285.68213327442413], [137.52413157674542], [138.77099483097822], [171.32958157803168], [230.4950211475794], [173.59287205616846], [153.26852158790743], [164.0406949416499], [173.50404127952606], [168.97297925775328], [287.92301633166784], [145.48057932575995], [155.72530496595067], [198.28174126748618], [248.18973869734418], [77.91668985918996], [96.30316788924527], [76.02725577067746], [86.30335523506272], [51.97537821108354], [89.13993520389005], [0.29850159105254154], [7.866315246055072], [20.72099767298833], [104.59600601845479], [47.941664386107995], [48.393788761254314], [45.12979119979326], [58.519710454505564], [75.80593541034489], [86.08673372732346], [56.28294351010449], [58.34279651907806], [56.43629938607993], [53.45714689644751], [9.886200936022622], [27.437273703356574], [54.81345710043033], [54.48691601333921], [51.63052714451192], [72.62701943854188], [73.2479075903971], [78.3820805692426], [56.79907391858568], [34.1202440027027], [60.288258915938734], [29.741878569153002], [46.53761615522169], [4.575533911849443], [5.571299137282917], [-0.17762940254975257], [-3.2860368723550484], [6.470288074093355], [13.98516596307688], [5.350801030384843], [7.335281311356492], [-2.459922955110405], [31.18190026853204], [49.01196678356485], [45.426784897432626], [39.72153544690694], [39.3520454105215], [33.929106465782226], [29.48460772644444], [32.200822658240014], [34.36489105831294], [28.20403044485142], [21.66857927586858], [21.30954411182858], [21.46816188962041], [59.89987011904498], [57.86743848968944], [60.52405541076164], [22.389848533242652], [52.71134341817512], [68.22992969542692], [47.94948824770465], [65.84216658987542], [19.812622931410964], [29.30293084479959], [-1.398225894274105], [0.4190050952801692], [22.702810527505555], [33.51205081556331], [15.265092158423982], [18.04791420259219], [14.00873327464576], [24.095163395432923], [48.2691905404521], [66.51621607427681], [24.219153651673054], [27.300903659588073], [39.85525343877803], [17.984422480872784], [24.66379653304478], [21.397891187008206], [24.76448211339155], [39.61561112371753], [52.36407284841902], [20.490254018539684], [28.548357580771903], [22.227272591095826], [42.50853867332725], [45.73495651428335], [53.48399798015894], [19.741952277155544], [92.31781624518824], [150.5744712077534], [81.93262460338858], [136.3576910861918], [43.048267323737434], [74.69374115327452], [54.03904601609908], [76.5976531888762], [46.946112995080796], [76.29648160244464], [31.450495871193436], [39.010957115409184], [33.1408274563971], [48.870197909644936], [82.72646589764496], [137.37672680240289], [44.73739357813233], [63.138327976165066], [88.33993703758657], [26.987816436632553], [33.29876899498582], [48.61070297584999], [33.06675195961705], [41.41000339770272], [60.15976918488826], [33.281114774763026], [50.93863915691937], [66.65091008099101], [39.90592347482517], [41.83488895879311], [55.0877873460965], [84.7611961163228], [123.7034892609315], [26.771223417179204], [50.80929666451893], [47.48083184985354], [60.707484420228724], [35.59370075383351], [47.68889055008451], [-7.067356183366687], [-1.5776311281029776], [37.89667333816008], [46.74760405344368], [32.15503936214073], [35.757950653312875], [31.702478341783156], [40.91097022645899], [47.513824689940435], [61.539552279410216], [41.6116585180397], [44.41853221482095], [42.821120274304654], [30.496825720247752], [29.391399234626512], [32.44602304613198], [32.36551550549957], [19.949819284872092], [28.554547596773165], [31.95979488967279], [61.89074861055707], [63.001804639678], [63.26065841223698], [32.20529146455779], [180.59426681903037], [257.8296002043633], [169.87639540110635], [232.2268798720205], [109.55123642663995], [116.70552897613842], [15.15914365576038], [111.66599170271765], [121.0011037769331], [106.36980956005742], [107.6902258872485], [104.16701752775862], [112.71247766252971], [170.13635994389932], [235.06149862714696], [101.231933624062], [91.31862009555452], [93.48237300927102], [84.38062533015717], [95.83445938039847], [83.96508917230263], [18.698346273387482], [35.10499842405565], [63.82342682708049], [92.23293342357998], [79.34455257101084], [111.69699426207676], [47.15217517147494], [87.20779617421704], [42.767298064639775], [80.35672761921366], [13.618002837555993], [23.154397654589367], [20.868136687432166], [27.306604069285356], [14.324572375172158], [26.246395900864147], [13.68014510957881], [15.747952498680899], [13.012569025210446], [16.165342916051113], [42.94323839245393], [80.54832144938067], [20.416754852937185], [58.60804760418495], [61.64247435060792], [7.429422408234276], [16.535921908019958], [32.4180958306018], [13.789553050360286], [35.438901285593715], [55.153942623191725], [29.80690454874301], [50.84383393476224], [96.25494328859835], [11.446612618639326], [22.29598350494783], [34.62331507398996], [31.238530707479985], [78.05295048004712], [60.98977496863957], [12.47572232301748], [27.243474531126683], [33.83689511093487], [24.03124306293817], [26.160849183681318], [35.43796390004516], [50.06631368698302], [31.456641327160604], [44.72822968254777], [37.42395596613652], [51.18047855789955], [26.460690757062228], [31.198063085075994], [28.607014071000567], [37.90932008518037], [23.82751506626119], [26.22674166030098], [65.72621328045982], [76.85104465633498], [56.99755294631224], [30.74832077427932], [33.074208784843314], [32.92556982476234], [48.72459924997932], [55.084138092387384], [47.20977512043079], [22.040766471522808], [21.030841157844325], [19.510904941728864], [31.73126836236415], [34.44182034961515], [32.0270481854768], [78.62373648620542], [88.11977589596692], [58.07115852827752], [23.515438956630728], [13.36487409664231], [30.020998681363636], [11.474291190665117], [20.878020393572605], [19.268044434072788], [32.59464305740582], [11.839933840755439], [17.47632186022867], [21.996681547364506], [31.067943864035207], [6.731486359039207], [9.90058033343398], [23.414088871300592], [11.432575251774221], [21.78399147024588], [54.89003668595807], [50.51463480310808], [38.26640455416504], [28.351403164475496], [27.89906064252671], [26.78402496203693], [44.27915435504114], [41.058443545493745], [32.46206437008199], [-0.2499411429124887], [-1.6977036368224794], [-1.910754982204781], [28.693320435676373], [26.712287955082868], [26.77154987820569], [64.67980350452903], [63.10213980326635], [61.43134518649095], [6.325506377861594], [44.33579649852422], [61.940876516605286], [128.4217680473316], [131.6029934944295], [115.44124255138607], [131.41746334253773], [66.59262778399426], [73.42054677495125], [66.8843172647836], [93.01246955081277], [68.9826680081659], [84.88171040656961], [148.31593264570324], [67.33834374758197], [122.3403538493436], [101.03151416606232], [117.36810948658092], [104.43003809846368], [170.25703985226644], [29.71561217714175], [63.077190717123514], [32.64720239217783], [98.23126885935964], [32.359508052580175], [43.17833434842892], [68.31808504304358], [28.58471940669415], [35.57388886837003], [43.44999397387216], [202.89298161866606], [185.27672451090427], [143.80546544534178], [162.4288330062181], [229.9446425335973], [318.41082677396577], [173.98183175236545], [235.47601804357012], [288.52108256438277], [31.432306383678302], [22.819777085957494], [94.2697150020572], [31.44437907969131], [46.91829050663399], [87.65566531781188], [33.25537529957175], [44.97109826824121], [72.36808213498695], [36.59070985034684], [239.78307975554318], [214.0818697113846], [137.4146042709551], [27.62283654266151], [77.34444346681602], [74.35334984456145], [70.67086304018359], [122.52325292176474], [120.76921512349786], [112.57865258070184], [125.41159325688099], [51.284195969122294], [54.07446917570183], [92.2575805300271], [39.56964214836315], [37.43026990008828], [0.0], [39.624552972242306], [102.64584094019614], [67.60463640602681], [82.23258967020398], [118.31212639321724], [182.2834902669253], [169.09413394756257], [138.88332829166671], [168.6629979400245], [219.14775764908896], [287.71483200024335], [173.2753636650063], [216.9759253095074], [205.74031187176053], [180.94583220851285], [127.79742603245523], [86.9161042853195], [303.015120052906], [322.6010147192401], [230.63717795137325], [275.6468491820531], [267.0540989831174], [161.74638612401662], [95.20600363048162], [125.76635690931083], [32.87910183109065], [88.76213759398209], [28.7074556388228], [95.24518384120911], [139.96088618791177], [32.09994777854548], [52.21549801053149], [53.28633208556557], [56.33053406249267], [248.6127730861867], [267.4255196512724], [36.517225780059654], [75.24032914217487], [74.88658222092774], [4.4763531747229095], [84.43964436686798], [123.85386726797338], [26.326128645061978], [41.81414393177868], [44.962112544194], [30.476349244083256], [220.63340169495976], [251.90749591292985], [30.05014886084296], [47.32828062117698], [54.26507824858867], [93.33844175548819], [119.06773349696847], [138.19898802871649], [67.34785745043787], [74.69724184205808], [111.83978021211804], [241.3828911516555], [236.59543126964147], [78.38205708450708], [77.87534590251815], [85.91155359539952], [76.20758969694656], [111.31076820595875], [71.01224640999523], [92.5249053009762], [81.07047702177917], [129.39822799480905], [9.97208170047403], [56.421626170608725], [84.67486465395068], [160.01773353322386], [58.86834196306706], [61.674282630488676], [68.11893977127673], [75.79545260133352], [71.3132836507761], [96.26561212974214], [44.053623794056314], [34.9102102280658], [17.97408306008785], [52.56512939939202], [45.329758261002326], [33.887550776139385], [-1.7634862191406684], [-3.101272794968122], [21.186342959964133], [47.59112424476219], [39.67233166511308], [61.63491467225721], [66.98666835008677], [90.7492328883725], [56.236353815505026], [42.184302653617486], [61.417124297940276], [35.74402420271693], [52.07632312128243], [14.947786486245668], [19.413639054775768], [-1.8078268649429658], [0.41956949158256474], [16.94917031560775], [19.09416059505573], [11.099322930760893], [10.210012190482344], [10.442125145750172], [12.731266881068622], [36.83053682378179], [54.33132937727509], [7.79757830262795], [8.681894056420852], [3.51189088510929], [4.60508302546185], [6.51623383188934], [12.99436383712942], [26.38722371160953], [16.36476900900522], [20.720465470388092], [30.79605768594861], [28.257787245304655], [25.304369541903505], [30.597861458868056], [12.48294842440238], [65.98539131399521], [68.90831662519278], [60.9255297543918], [68.96023245797463], [52.83864396210923], [65.68518033132784], [0.7500894370566887], [-1.1811857872377387], [53.712898600717146], [84.14542182382397], [44.52179648010549], [46.08438427192194], [48.28022226760439], [51.46171513673136], [61.64239876830241], [69.1188341840263], [18.315205925284452], [23.78808299816785], [33.95473294069378], [25.4197627345432], [11.206730670855933], [-4.909052453668647], [-3.217189814969991], [21.632054810444878], [40.53968949297355], [37.99611694271007], [21.002365784947614], [29.05837770071359], [54.14070838200353], [43.040104896428836], [122.13097839049608], [168.1170925339005], [113.64578961467613], [160.25397807907876], [84.87422006544749], [83.70496736021367], [80.20645771109331], [89.73257734405733], [87.34249732561035], [76.6777662399232], [69.72607142789889], [71.29938258117514], [74.03215599207259], [81.69358443086921], [114.62548269869802], [161.02981542825682], [82.40975672284851], [101.03713662532657], [139.63748240549418], [62.33594539183044], [58.7490117296318], [55.24931526558259], [71.25600702332393], [77.69168898354285], [91.1206337827454], [64.2582368768207], [64.27715969628262], [75.2791410995334], [70.35652968629864], [75.9839262763512], [81.66259347459179], [93.01672849503596], [121.38238934348234], [168.23286401806422], [68.60512234414587], [65.88163184344502], [60.583537952623274], [64.89825404988567], [59.02138177269037], [53.04627054226373], [-1.8122872270857007], [1.3554548696219564], [59.9986144181518], [54.85754771229082], [49.04772280954417], [50.58465922180678], [52.98424170333033], [57.17533704502172], [61.18852535736354], [65.6367014021723], [12.418893527013697], [17.33528569507321], [20.722278416966287], [42.928090293411756], [38.91748807406555], [22.576112346007765], [25.591245902051387], [32.35638025454117], [-6.070690278724683], [-0.932734199847336], [17.904972095415275], [24.68432971443478], [21.3667450176314], [47.61892935605436], [168.99327262088454], [260.6766361968655], [157.28162224060745], [233.15507016655974], [115.1510142228687], [129.50287900335056], [40.04988529382489], [82.4508373398613], [114.64205542686229], [142.12441739351186], [114.91048455303005], [115.29280683307763], [115.91426708058133], [115.66516751919234], [158.97769882049218], [239.0122756507771], [107.67937533888252], [99.59168805785853], [93.50832874451113], [70.52428556104245], [101.73165245914412], [86.5709310484164], [11.161339436071408], [9.728738341304776], [34.48460098726044], [96.17950574801652], [73.45333624876939], [32.12237256164799], [114.71176345532861], [63.30080166976901], [98.74129916794874], [59.23609928631441], [93.74462688759411], [34.46393159621157], [41.45345385534851], [32.917428007300806], [37.836712666669655], [35.858483878693875], [43.219195735761865], [30.452340195598257], [30.53483625911497], [31.24386985973689], [33.03987479899204], [59.53337993173756], [93.73427976527744], [69.36886331729966], [71.6138731583316], [12.31951175444051], [23.56368522230218], [27.73661232794483], [20.757267523665053], [42.87539287306915], [47.56484771440324], [15.922482531650273], [48.37333831868144], [65.80375880437624], [91.8687989715773], [25.016641716822978], [27.362714453538374], [10.179657982458508], [85.90837023828307], [79.64431127152218], [26.31914578305693], [30.51582579901839], [32.88533918167902], [34.1704402757127], [31.87303304702247], [29.800656392839663], [46.84778706454012], [50.3890480309738], [43.8223317802599], [50.36701155210833], [48.18995794427278], [47.500851763599655], [37.45023668078399], [38.96984559089771], [40.57036453254492], [45.37537311519688], [31.721819452693577], [29.52541689760168], [66.85745731659229], [48.50345477131192], [8.374216303462134], [24.733274530655425], [20.065660975085247], [9.285744407687147], [43.27266536554302], [31.80356217410352], [4.002600922848704], [34.25925248591518], [31.71400912248308], [29.044083638513044], [22.854285218994494], [16.583292540219517], [2.4250110532681637], [79.59480646361676], [58.74834030746224], [20.78646438690847], [36.05509892707006], [25.439286865482863], [29.756720037783676], [21.63652775934481], [20.21399698337474], [40.41680367576184], [32.359774797356536], [15.774188290563371], [18.74783218329278], [41.56566399790811], [26.683335966304416], [29.857091541955903], [33.40392341723439], [37.36468422531643], [45.03675489256164], [21.75090904300456], [21.065359046577885], [21.464311263475516], [14.703750911668164], [10.199033151661567], [12.643626373101517], [12.26263014800981], [12.042503397193283], [13.130354360333493], [9.180001640851152], [10.751576990803326], [11.345802258030023], [10.115317263542185], [6.953876863514681], [13.249396458274155], [15.098569494431363], [19.342871877594888], [42.129908701114125], [32.366752343568734], [21.148663471396937], [24.53102594276758], [207.0308236501584], [180.46133452792617], [192.45725222690942], [173.2716421114709], [212.64808190670428], [235.68288640960088], [276.2458324500637], [143.431887390178], [144.6449787813107], [146.0823062854318], [142.27468497746014], [37.48134106698351], [162.8451471589336], [156.08152616648104], [310.25887893828366], [304.89840801053174], [280.22163177497924], [305.30762672583165], [284.11565668493887], [304.514333992138], [264.8856805966071], [186.4232841044454], [188.22346880866576], [171.94050423300754], [148.82986111415735], [152.06938378864587], [202.53321048942587], [223.36482888293042], [265.11496545565046], [134.52328155125198], [127.49114643501665], [126.4923406486095], [121.2334047608472], [117.89376769539732], [162.09777653490391], [154.91041487329852], [173.35018320533857], [308.821576331058], [301.5513990466544], [265.58726919463106], [301.11606835922674], [275.8057135825152], [295.3004505661297], [257.0298008641506], [173.42936359878826], [160.27231648345315], [145.22208734035505], [152.5034631499443], [138.30348869834546], [117.60138984954908], [191.9793636260663], [210.42515952527353], [253.89540168689402], [129.39270400207707], [122.85163886853323], [120.20597766010023], [108.93523223851525], [161.5896311254982], [154.6274468530942], [163.16553042476474], [305.0196929175801], [295.05117069620053], [254.62010326897905], [294.5426346911089], [174.21687714428876], [173.9068898821527], [170.41367374316874], [220.60950852089246], [90.28030300049736], [105.68264786026694], [193.27003394728945], [92.9805712967554], [192.80984129255907], [87.20157667194094], [96.53628085163172], [167.02620046915158], [87.91125580705145], [205.2396731490402], [88.27444088162433], [100.09964634668792], [180.06689725707864], [105.96418561151854], [177.02193154922733], [86.9295531498993], [89.66346680061899], [156.08110397847773], [87.49669285239956], [143.8759086358461], [93.41987910809394], [81.40281588612098], [218.85414821782703], [150.52524451578648], [165.22678458223342], [207.29914545202735], [173.50394762090693], [236.71172224423844], [161.80479071561984], [181.83369665252334], [224.66685947851397], [189.72735398315984], [278.0296211323245], [190.06920132500406], [200.56723442286878], [267.98192701289406], [191.062907798201], [142.47103324562102], [99.8219298934294], [87.65651582715935], [136.05481519559842], [103.01506317126571], [138.976254696322], [100.23378734185692], [85.96259823468618], [131.44867072210593], [100.80258535614159], [131.24654865008225], [93.16257089483732], [141.39326769985075], [74.5182207625125], [67.83090502429636], [127.61094314616204], [79.14070949372052], [134.37458062950307], [73.29831786753164], [67.56294127291092], [122.38026624031941], [76.68443658465591], [162.78028434625375], [137.83107406266956], [124.34118164095989], [162.3112777546038], [149.66590352180006], [156.37048071080406], [142.34350803324648], [125.26709675610621], [155.1116212034694], [147.70842517869963], [192.7433595601092], [140.81708165868048], [141.85062606629842], [148.7232698250454], [310.32885103381], [246.3117849052889], [258.9388378716532], [309.09929937066596], [280.08563745669073], [305.74565877859874], [226.541490102459], [249.478303585753], [301.41345501378476], [261.1034971941233], [278.5133082569035], [185.63815268039275], [208.80941722467566], [268.570052691201], [193.5045510099728], [306.073557811352], [227.79270949483444], [246.66033323030763], [301.6307953925023], [264.3223014516371], [286.26313554629826], [194.55919507547665], [220.0886092776454], [276.7041145457009], [229.31966866455423], [302.49864986982953], [219.3042615294879], [243.02671351302016], [297.3092893933595], [256.38917639659184], [268.0613902862477], [178.36247940263243], [213.72206404833182], [256.96245498778103], [210.19913111167756], [171.36087303120428], [214.84928614362119], [85.86271429886594], [84.16307523798619], [176.32364448614143], [83.859151926148], [205.7871212519547], [85.15214902377505], [89.08691134956975], [165.342331295846], [84.02374626295418], [199.00059644028622], [84.54368951297245], [84.25628841555984], [158.74485469249794], [84.45413844231538], [181.38815740064857], [86.49794070795954], [82.21380826228776], [145.45441324704558], [88.6023754013201], [146.13698875156408], [93.36587786468297], [80.01431841915584], [216.57077595303332], [137.09999496855463], [150.63871929617628], [198.2698130180751], [134.36410099245984], [240.620515768337], [153.28679975590052], [166.53523644149507], [223.7003546565772], [170.4409697904794], [278.9601109355803], [173.95169268818722], [190.75185667452223], [261.61103293846], [197.5783560083213], [144.617295553113], [96.60905977503023], [85.59322059286103], [132.34219720241234], [103.3942206585172], [140.62310249174536], [97.33259772609368], [83.82902745281866], [126.18175349716559], [101.16758327731011], [79.62253415168757], [122.81614854729659], [92.94691661288944], [142.37993678904172], [72.56983799717055], [67.0927953616274], [119.18431352898614], [77.0679268940711], [139.4369242867087], [71.58165679380923], [66.89774107817324], [114.91620407386873], [75.27996574572907], [162.82543784393175], [131.3254218856174], [117.19557839173466], [161.94098519826207], [143.61032562499534], [156.54948253554923], [138.10698876882338], [118.8573919594014], [154.74897544201596], [145.3387071922768], [191.20581439611433], [138.59455038465885], [132.5777868867122], [168.93590794964527], [143.87590144404567], [310.3286824642322], [228.9277324895088], [239.86815718817732], [308.14848100047175], [255.61290539958307], [306.36962846365947], [212.95957182557558], [228.71253178486978], [299.88881287524646], [239.57947643810562], [287.0422692664277], [174.6909234913518], [191.57388604058642], [262.7891381879179], [159.432828595869], [306.7126665624335], [213.82091939102565], [229.31898130481903], [299.46172338273146], [240.4253770007421], [287.8622949616658], [182.89763090908858], [202.56154609605926], [272.78476498980444], [207.16076232680848], [303.40260638582623], [205.55602689180787], [225.06889496268278], [293.36503509357277], [231.5281077423082], [275.301527121071], [168.54122054502415], [190.15751383121614], [253.32240916371236], [189.03578707690275], [133.37848411776153], [187.37760173226712], [175.09254775021506], [85.6468045426232], [92.75782104165673], [85.56429095412206], [158.05107894136896], [84.99802247531687], [86.46590614290652], [85.25908211780691], [178.93641044972276], [166.20562265418206], [85.2237266926778], [88.77632591406183], [85.1838283289141], [158.25183319211587], [145.63707834027497], [86.47784370175361], [80.82547275153792], [87.51247341430795], [122.39742128571099], [205.24245582849062], [202.18305189473176], [140.67952582102706], [144.83428508228076], [141.82982431597685], [228.36149182473983], [225.19628530059853], [147.99067936490525], [161.09889999583268], [148.53132526959433], [268.9571599120334], [260.9564238168277], [170.19467803525757], [184.23411563532252], [167.06268750287433], [139.2240189706448], [131.95274932277738], [94.04161091922042], [83.61948328736084], [93.65236232792888], [136.2523859631017], [125.78888870332236], [95.53500880700412], [82.7335139285397], [100.87175568183237], [132.54186691767978], [92.9811561855472], [78.76542378132628], [93.32865907502838], [128.98889759875698], [118.4305313041459], [70.8325203972363], [66.16631980738971], [76.47692710267425], [123.69820869781708], [114.21383656280582], [69.99056292520747], [66.00287580325273], [74.84932459218196], [162.360520451805], [161.92771735980682], [127.59874464089846], [113.99372297962138], [142.29492610030934], [155.16139799794712], [154.7351797180167], [135.56029493095184], [116.18039483761684], [144.74133915897426], [171.4859686955969], [168.5178063682023], [137.15632478937556], [128.62743157566536], [143.2477493103418], [309.4149129753839], [307.8038907129684], [222.20901022979027], [234.1149826479043], [220.08994665995047], [302.68672339530883], [299.3534834474126], [204.43824689763196], [219.00318611946207], [237.8590544647411], [270.0987799334271], [261.8788245128788], [169.06131758537674], [185.7932696105156], [167.27764873785145], [302.30697979447456], [298.9340854250376], [207.29050421011942], [223.43487026979608], [238.31361588848688], [277.81657031272016], [271.9866745271202], [177.09397050117536], [195.9706309142472], [204.40490700615223], [296.6973342671561], [297.470882529599], [202.45123640263276], [219.30676424878047], [232.91402942517553], [579.135981131913], [623.9240294084649], [473.8099579696088], [461.1962321697593], [566.6398058987933], [164.01309824281662], [146.95336334613583], [163.69846671530541], [138.6836733234485], [126.4235800825], [135.4605688984374], [137.15414525987745], [138.10257605769644], [148.74263045269447], [133.22434582676325], [122.44054789888992], [119.56390362769417], [131.75198949161268], [123.86382405939263], [121.3012844035635], [119.89717274202468], [124.97946142510362], [139.39706179055474], [113.5356940420492], [107.89827540770035], [117.78653305945782], [120.03568276746799], [123.27187232097552], [143.25980979377863], [111.38578230865066], [116.96283344508298], [105.49814360965327], [115.90368764478093], [119.9658175503996], [118.25965084504456], [141.5425434359687], [140.30388902199041], [157.57573529996358], [127.68257257370033], [124.5756197966182], [136.12440579447332], [133.48451964597535], [133.79316404203345], [148.72016441788645], [126.81994471647415], [121.49022224835184], [117.11344656607626], [126.7762306291678], [122.94233052152855], [120.81382068405263], [145.74643999819273], [135.634270447454], [153.52450858633154], [129.9973449765553], [119.92389121095994], [132.7927481303682], [131.882875725132], [133.80259362112625], [148.37796928783163], [123.31987654511664], [120.78617827777343], [114.05131273823194], [127.04489699560823], [122.23585425072228], [120.62426460736468], [110.58195274826247], [127.28085145677835], [147.6260654313294], [56.97997091924426], [71.96880378463483], [90.46722324255643], [69.26694501404012], [64.1131172431923], [94.21051445436153], [34.404309312532014], [38.420802755887514], [46.26961263962415], [34.64041047387568], [38.7792816165832], [46.64379918250774], [33.33034794207542], [38.456367101040186], [47.764377085527265], [95.68781411213844], [98.75783953609209], [128.26050262695097], [57.63443009260603], [54.464068748640116], [89.87217009171344], [90.30001202933528], [98.38982843971434], [93.02683643800819], [93.09687608400891], [104.07159087672075], [105.19715000134653], [106.29467645336864], [122.74130027822802], [142.30745209793028], [57.359688511153195], [72.28150264214264], [90.57016335008873], [66.79575079863946], [62.193226272554874], [93.68768542906709], [34.02343081089505], [38.18197523891716], [46.07918366691403], [34.363901905943266], [38.45051394520395], [46.63813785356994], [102.79197401391546], [118.84017300433891], [137.53120019035558], [57.631431925676196], [72.46338229369056], [90.5543181031808], [64.68783453846689], [60.63698336307791], [93.51173110042633], [33.66269081129752], [37.96199229752794], [45.87450528801219], [34.112053068035614], [38.365286661528344], [46.616670115462746], [250.0471113197836], [179.33077556487237], [181.66937514445152], [174.91014944895286], [179.80246458134198], [176.64518304349843], [172.74497108855115], [174.54262902812312], [178.38092629373176], [236.2583519928501], [161.40129146093102], [163.29624388232529], [238.96553538065268], [163.15545553896777], [164.56200835443246], [163.89720688348825], [132.887972891513], [133.31098915233966], [192.26299091857825], [189.70297669447504], [179.9849781279238], [177.9806664258932], [179.99295902027708], [177.51665067701887], [158.52659728158204], [116.8713263408505], [118.1457560796362], [162.77662507586422], [126.73641448348116], [123.50517215852624], [244.66887722812558], [175.14974231008287], [180.50437224376577], [189.10698804546095], [189.78287544863358], [184.4006930272133], [174.09824494919184], [178.6804472078187], [178.21701237743548], [167.17210956104688], [130.12258448122213], [123.56919875043116], [168.40202954894582], [133.97225516855875], [125.29996417103179], [239.95288076831122], [171.60820577027943], [179.35391245076102], [190.11405152378774], [190.05857812544164], [183.06789304042704], [175.06733014167827], [178.9461919406131], [178.28174029745304], [165.70015759853908], [127.50200882092669], [122.2855963750122], [167.50510428172728], [132.91310939047824], [124.96173277326504], [101.74278565871273], [101.7909217783671], [77.06305998365968], [53.88047428899464], [49.04175649437966], [128.66082590690243], [221.27308764957013], [123.48645334936168], [130.48659224007113], [152.7534936925337], [160.29473898203634], [125.59946507528579], [124.06931287984834], [122.38546528669563], [307.56040095034507], [151.60749881779498], [162.84254284033602], [215.76538409792184], [194.9113222938579], [113.87872551128997], [115.20555464307216], [131.31254312659178], [134.0364726950048], [107.02264841408041], [104.24836325726805], [105.54887320869705], [312.0335992736622], [148.16203258018254], [153.13739225680334], [198.62233465767238], [209.97618947659373], [118.70369027126875], [121.9229223476262], [141.41763394888935], [145.5947539388972], [113.04030471854033], [112.41338515338538], [112.38897559388876], [310.19370171502595], [148.92983694644576], [155.97906815851806], [205.59607871723185], [208.6555878928706], [116.61364901915331], [121.47372954931782], [137.6641371760328], [135.75997912403548], [115.98382942619195], [111.86271420234796], [111.2805917759432], [312.12755914468397], [148.92332028152606], [155.9630284865159], [205.07863381429797], [101.8862978859096], [184.60552862595722], [110.31813486451097], [112.72494700692167], [126.49752887180965], [127.22803474150999], [101.10461033396521], [101.20450453013831], [102.81154961537695], [311.7695350284551], [147.953508], [152.1526617045573], [195.8349013129134], [106.93137213595311], [197.8489817302283], [112.35889775967074], [116.12732375109293], [131.52874009639817], [130.96956807827618], [105.72257528161634], [105.25130610708935], [106.04602896816573], [312.11056743815107], [148.26155703002397], [153.62302058533768], [199.90298173537496], [77.34444346681602], [180.7447135408561], [107.67336171827775], [109.92239746445503], [123.06040720485892], [124.14040057796228], [97.10744907459794], [97.72716642765694], [99.99724409407096], [311.25527394010123], [147.75933734504923], [150.93919940894332], [192.23810453399005], [64.1802425747915], [67.01919355813801], [72.9357506257604], [63.040396480057936], [64.59700364013165], [67.84255092763074], [34.65168807800954], [44.02775997883167], [105.96547886153823], [36.17500840152564], [35.1415705127038], [112.43949634872531], [34.82280428187747], [50.27138025496738], [109.79516441160013], [49.81287500352057], [39.670591468131015], [74.34630770067915], [87.07951602230028], [144.61291924502706], [42.688975232349684], [29.778645195266368], [32.52683184798542], [45.95656918293656], [32.43642751207148], [38.51850005428053], [29.121054616250227], [57.752444755039136], [51.82698839683667], [41.609368116418004], [59.85548896827268], [59.48034027906213], [59.25780521366778], [84.92642771625724], [98.64493956909483], [118.11415972112285], [80.74876728349103], [84.00223179818084], [89.4031353364007], [86.67104215336633], [124.19759346547588], [154.09160844136065], [88.7496776139223], [91.822909294178], [91.96196212131754], [76.9900330669683], [99.34000977748592], [111.50938591460368], [79.35428370895333], [102.98854755512829], [107.26007085471339], [24.80880016381004], [36.339463743710986], [18.819251132246315], [81.00350939871394], [59.71341351055921], [97.29523315090896], [70.3645330978454], [81.98995917520794], [115.75931120217959], [83.62408800083735], [89.71034473291918], [98.78026066813126], [103.80859587541295], [133.78758447854176], [197.82047087560193], [48.2589429300799], [37.347605565378494], [22.028592936453588], [87.53456841344693], [116.91690308039607], [158.52477468099002], [66.1956249292282], [58.02556803993812], [115.32313781953391], [87.31212205355881], [123.42610073928655], [151.94643045155425], [118.52622888683672], [101.49311658031017], [150.86726503897148], [51.195446045814], [59.66936669011033], [116.08414204449473], [83.52942324873197], [110.12847517300771], [155.55612833692993], [88.20963453726077], [101.32151780056381], [126.346523235716], [6.476054845619975], [7.282606813363184], [7.734861000175047], [59.05332036688735], [56.29888622933012], [51.915820577294625], [82.21110926420553], [108.1874712612059], [111.30944142742405], [28.68894981306852], [49.494766301482954], [23.02256110807056], [5.83707564171699], [5.96204771621361], [6.3206309451981575], [59.19319301541162], [56.2148032290805], [49.23714066772367], [80.35482675935872], [110.34417189049158], [150.3506037495375], [48.269306318404986], [37.704439362957785], [23.58624195967583], [61.524283968769325], [79.0980665236486], [95.58442022942106], [74.52122605715257], [85.98861998689623], [45.287584068346796], [83.69048646820362], [-2.1017027673402504], [1.5842765104958096], [49.36648970419169], [100.5168793461479], [45.89633347143269], [45.69829408350812], [41.59384772590793], [53.681132395129545], [76.15132965157444], [88.9442432876641], [55.97086881726555], [52.295755130943554], [41.553924654503476], [66.53418623376886], [60.72055198178991], [40.55253969397825], [8.037521805855057], [26.811970965562516], [55.84936063100006], [56.173433905139454], [48.82941053353599], [34.69789181020433], [64.0674579762374], [66.87468260577282], [53.51400153580734], [42.702686362300526], [75.3701004004208], [34.26004135141054], [53.944117008268734], [0.015716748072412856], [8.825448066220023], [6.009050036358477], [0.5216673873641089], [-0.8772809792220996], [10.59074144491697], [16.185640469138498], [9.81393898331431], [9.805058448171621], [2.414490940110672], [36.604144766481504], [60.076896290569806], [45.34242181819899], [41.332085350899504], [34.13156342356418], [30.517551503037467], [24.720143894416296], [32.75939814634621], [34.76688614023541], [28.29603074223302], [22.806280575549696], [23.045319645963602], [60.24010912126182], [60.22409197694657], [23.003621206626132], [63.25291190806214], [66.9011931932667], [55.61848255106414], [68.17884887445561], [27.613593027931657], [39.59111577754274], [-1.221288558226564], [0.4666405329019998], [29.674822168627596], [43.64884115774688], [25.982523785582604], [30.22784404017583], [23.826175794906487], [35.36800794858594], [58.15236759676381], [68.28546326312846], [32.30270335825766], [35.31909067118482], [25.060126768554174], [23.6502206916468], [20.64968433929564], [36.39890268522611], [50.562414169648996], [28.06972246269619], [28.649435704460764], [50.13589196979894], [54.378075825037], [27.736956637331854], [116.4965203054686], [165.77091697811534], [98.00703733177293], [150.7917769763572], [49.694177095075574], [80.02209503685535], [46.71452995030708], [72.93482880883855], [52.943782417120225], [80.76531176630618], [44.17914344683183], [52.72708054523069], [43.73740241192812], [59.16295470341749], [103.13386347210705], [154.7655774865411], [54.12520698432573], [72.00565648731691], [92.94766410606651], [35.68939803599995], [40.43266490893055], [59.79328156043136], [43.01353289149589], [50.94487762688592], [66.62713455994538], [41.42852604527589], [56.75193438267032], [71.96506634748114], [42.316519940435875], [49.97394001543383], [67.1774284617603], [66.0145134482361], [96.24833897128507], [130.74635694157422], [36.158757348619424], [53.9529252774434], [46.09058305873239], [60.04080717098581], [29.534700809231147], [42.762483184725774], [-6.755186607857395], [-1.7355938212907704], [31.387288707039268], [41.57682310447604], [29.4984881685003], [34.01147900048479], [27.79732125774861], [37.870727779268094], [48.528775525110916], [62.87746087879477], [38.95191834607627], [39.253372256821955], [27.368919569459017], [26.35956911078674], [24.672274533197044], [29.306589876231918], [29.014814560115415], [29.026758751235644], [20.65255333412804], [28.8715917611604], [32.36155588144883], [61.12043862923492], [72.55325266897849], [28.775084266553716], [164.47692795952602], [242.42281493120484], [140.55154693990755], [200.6009896597749], [98.33112770619533], [124.81017822921169], [11.612055590478684], [33.86486833503755], [102.45272746782685], [135.48533493721118], [85.31773020342274], [92.18191316998463], [85.1516497621127], [104.53273690700858], [147.68281753895664], [213.80192147559706], [79.89424015139973], [72.60965387041081], [60.66322388873691], [73.71235129690146], [72.31260607537995], [65.61720208595351], [83.94701855234862], [75.51550939148864], [53.8645372594108], [18.833107845370883], [34.33882121334135], [67.51272703447695], [79.01147019045588], [62.237541772657465], [49.666704691698634], [87.98343045940736], [59.82757554188898], [100.25304295603435], [51.03062742008486], [89.85923787911462], [18.511658040723095], [28.873880289279846], [18.831483140453532], [25.801112780241223], [19.061339215159126], [31.74153212194712], [20.177567242978714], [22.82870882516479], [19.003876568860353], [22.888293600988185], [53.287034473533105], [91.99677056937014], [23.999522006946172], [60.68167346742346], [58.98775263484057], [9.363189189719451], [18.716713963163272], [33.032899365781255], [16.940084586298656], [38.73289128333805], [53.632584044026466], [36.011723333479594], [58.28460637126813], [104.1366232574398], [12.011973325043819], [24.162879880012404], [34.01818281819434], [35.63817029715472], [78.68968926783798], [59.89766802038032], [17.746221645116567], [33.37042970386121], [41.189559603022055], [25.205035803445323], [27.079728902954702], [35.10349742856058], [49.919078637699634], [31.601438347853254], [44.73486736422174], [37.323326038766666], [51.05789259261841], [27.13064854026183], [31.43818381728685], [28.711068178650194], [37.86360647677332], [27.43412191845568], [30.608024183454216], [65.75363779609275], [76.96255671524212], [58.93600819484486], [30.77993363220189], [33.12052308308753], [33.179645922164305], [48.73094258055832], [55.116466886775754], [48.14735091093389], [22.089298150965508], [20.22085812907065], [16.90029247051458], [31.744040686607246], [34.43788550711087], [32.355413702906155], [78.6166836732693], [89.67600514178565], [59.63519529472243], [24.309894632066733], [12.153714598080569], [29.202690155147437], [8.692133543019661], [15.999088797226001], [14.304879832094308], [24.05736368989351], [11.788117151529939], [17.499772206560593], [15.956330833394127], [22.879619215595604], [5.6608978879072485], [10.37053090836108], [7.968234814366743], [17.470646824217486], [9.577971696319082], [18.992154277247046], [57.42090546564253], [51.12485515896909], [78.15340384961827], [28.96157258738936], [28.291972308928386], [26.623372753998705], [45.63882408494711], [41.031434697463], [29.938648054543556], [-1.952202501812101], [-3.8394195333540315], [-4.659136499554293], [27.847506558658132], [26.616474438170368], [23.73664957267432], [69.2851840905632], [62.255647388599535], [50.226948070278105], [5.51842863178357]]
avg_stats = OrderedDict([('R2', (0.771823494407079, 0.0)), ('root_mean_squared_error', (45.89988069094602, 0.0)), ('mean_absolute_error', (31.369644013893062, 0.0)), ('rmse_over_stdev', (0.4811262117767574, 0.0))])
savepath = '/home/nerve/Desktop/skunkworks/lane_schultz/learning_curves/learning/StandardScaler/SequentialFeatureSelector/KernelRidge/NoSplit/average_points_with_bars.png'
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_predicted_vs_true_bars(y_true, y_pred_list, avg_stats, savepath, label)
display(Image(filename='average_points_with_bars.png'))
